In [1]:
import shutil
import os
import random
import os
import re
import shutil

In [2]:
def format_line(line):
    line = line.lower()
    line = re.sub(r'\s*,\s*', ' , ', line)
    line = re.sub(r'\s*\.\s*', ' . ', line)
    line = re.sub(r"n \s*'\s*t", "n't", line)
    return line

def split_sharp(file_name):
    inputs, targets =[], []
    with open(file_name, 'r', encoding='UTF-8') as fp:
        for line in fp:
            input, target = [], []
            input, target = line.strip().split('####')
            if line != '':
                inputs.append(input)
                targets.append(target)
    print('Data read. Total count: ',len(targets))
    return inputs, targets

def merge_sharp(inputs, targets, num=1):
    # 각 target 항목을 num 번씩 복사하여 확장
    targets_expanded = [tar for tar in targets for _ in range(num)]

    # 병합
    merged_data = [inp + "####" + tar for inp, tar in zip(inputs, targets_expanded)]
    # 파일로 저장
    # 정보 출력
    print('Input count:', len(inputs))
    print('Expanded target count:', len(targets_expanded))
    print('Merged data count:', len(merged_data))
    print('Data saved. Total count:', len(merged_data))

    return merged_data

def replace_space(file_name):
    inputs, strips =[], []
    with open(file_name, 'r', encoding='UTF-8') as fp:
        for line in fp:
            input, strip = [], []
            if line != '':
                input = line.strip()
                strip = line.replace(" ", "").lower()
                inputs.append(input)
                strips.append(strip)
    print('Data read. Total count: ',len(strips))
    return inputs, strips

def matching_indexes(strips1, strips2):
    matching_indexes = []
    for index, item in enumerate(strips1):
        if item in strips2:
            matching_indexes.append(index)
    print('Total matching data count: ',len(matching_indexes))
    return matching_indexes

def remove_data(lst, indexes):
    list = [item for index, item in enumerate(lst) if index not in indexes]
    print('Data count after dup: ',len(list))   
    return list

def extract_data(lst, indexes):
    list = lst
    return [list[index] for index in indexes if index < len(list)]

def save(file_name, inputs):
    print('Saved total count: ',len(inputs))
    with open(file_name, 'w', encoding='UTF-8') as file:
        for line in inputs:
            file.write(line + '\n')


In [5]:
# test data copy
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]

for task_dataset in task_datasets:
    task, data = task_dataset[0], task_dataset[1]

    source_folder = f'/home/elicer/ABSA/data/{task}/{data}'
    target_folder = f'/home/elicer/ATOSS/data/sft/{task}/{data}'
    file_name = 'test.txt'

    try:
        shutil.copy(os.path.join(source_folder, file_name), os.path.join(target_folder, file_name))
        print(f'{file_name} has been copied to {target_folder} successfully.')
    except IOError as e:
        print(f'Unable to copy file. {e}')
    except Exception as e:  
        print('Unexpected error:', sys.exc_info())

test.txt has been copied to /home/elicer/ATOSS/data/sft/asqp/rest15 successfully.
test.txt has been copied to /home/elicer/ATOSS/data/sft/asqp/rest16 successfully.
test.txt has been copied to /home/elicer/ATOSS/data/sft/acos/rest16 successfully.
test.txt has been copied to /home/elicer/ATOSS/data/sft/acos/laptop16 successfully.


In [8]:
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]
train_inputs, train_rm_idxs, rm_sp_inputs = [], [], []

print('/========= compute data dup =========/')
for index, task_dataset in enumerate(task_datasets):
    task, data = task_dataset[0], task_dataset[1]   
    source_folder = f'/home/elicer/ABSA/data/{task}/{data}'
    file_name = 'train.txt'
    input, rm_sp_input = replace_space(os.path.join(source_folder, file_name))
    rm_idx = matching_indexes(rm_sp_input, rm_sp_inputs)
    rm_sp_inputs = rm_sp_inputs + remove_data(rm_sp_input, rm_idx)
    train_inputs.append(remove_data(input, rm_idx))
    train_rm_idxs.append(rm_idx)

print('/========= saved remove dup file  =========/')
for index, task_dataset in enumerate(task_datasets):
    task, data = task_dataset[0], task_dataset[1]  
    source_folder = f'/home/elicer/ATOSS/data/sft/{task}/{data}'
    file_name = 'train.txt'
    temp_list = []
    for i, input_list in enumerate(train_inputs):
        if index != i: 
            temp_list = temp_list + input_list
    save(os.path.join(source_folder, file_name), temp_list)


/========= compute data dup =========/
Data read. Total count:  834
Total matching data count:  0
Data count after dup:  834
Data count after dup:  834
Data read. Total count:  1264
Total matching data count:  658
Data count after dup:  606
Data count after dup:  606
Data read. Total count:  1530
Total matching data count:  807
Data count after dup:  723
Data count after dup:  723
Data read. Total count:  2934
Total matching data count:  0
Data count after dup:  2934
Data count after dup:  2934
/========= saved remove dup file  =========/
Saved total count:  4263
Saved total count:  4491
Saved total count:  4374
Saved total count:  2163


In [9]:
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]
dev_inputs, dev_rm_idxs, rm_sp_inputs = [], [], []

print('/========= compute data dup =========/')
for index, task_dataset in enumerate(task_datasets):
    task, data = task_dataset[0], task_dataset[1]   
    source_folder = f'/home/elicer/ABSA/data/{task}/{data}'
    file_name = 'dev.txt'
    input, rm_sp_input = replace_space(os.path.join(source_folder, file_name))
    rm_idx = matching_indexes(rm_sp_input, rm_sp_inputs)
    rm_sp_inputs = rm_sp_inputs + remove_data(rm_sp_input, rm_idx)
    dev_inputs.append(remove_data(input, rm_idx))
    dev_rm_idxs.append(rm_idx)

print('/========= saved remove dup file  =========/')
for index, task_dataset in enumerate(task_datasets):
    task, data = task_dataset[0], task_dataset[1]  
    source_folder = f'/home/elicer/ATOSS/data/sft/{task}/{data}'
    file_name = 'dev.txt'
    temp_list = []
    for i, input_list in enumerate(dev_inputs):
        if index != i: 
            temp_list = temp_list + input_list
    save(os.path.join(source_folder, file_name), temp_list)



/========= compute data dup =========/
Data read. Total count:  209
Total matching data count:  0
Data count after dup:  209
Data count after dup:  209
Data read. Total count:  316
Total matching data count:  39
Data count after dup:  277
Data count after dup:  277
Data read. Total count:  171
Total matching data count:  38
Data count after dup:  133
Data count after dup:  133
Data read. Total count:  326
Total matching data count:  0
Data count after dup:  326
Data count after dup:  326
/========= saved remove dup file  =========/
Saved total count:  736
Saved total count:  668
Saved total count:  812
Saved total count:  619


In [29]:
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]

for k in range(1,11):

    inputs =[]
    
    print(f'/========= #{k} Read data =========/')
    for index, task_dataset in enumerate(task_datasets):
        task, data = task_dataset[0], task_dataset[1]
        source_folder = f'/home/elicer/LLMSS/outputs/{task}/{data}'
        file_name = f'zero_train_{task}_{data}_{k}.txt'
        _, input = split_sharp(os.path.join(source_folder, file_name))
        inputs.append(input)
    
    print(f'/========= #{k} Saved data =========/')
    for index, task_dataset in enumerate(task_datasets):
        task, data = task_dataset[0], task_dataset[1]  
        temp_list = []
        for i, input_list in enumerate(inputs):
            if index != i: 
                temp_list = temp_list + input_list
    
        source_folder = f'/home/elicer/ATOSS/data/sft/{task}/{data}'
        file_name = f'train.txt'
        _, targets= split_sharp(os.path.join(source_folder, file_name))
        merge_list = merge_sharp(temp_list, targets, num=1)
        
        source_folder = f'/home/elicer/ABSA/data/{task}/{data}'
        file_name = f'train_zero_{k}.txt'
        save(os.path.join(source_folder, file_name), merge_list)



/========= #1 Read data =========/
Data read. Total count:  834
Data read. Total count:  606
Data read. Total count:  723
Data read. Total count:  2934
/========= #1 Saved data =========/
Data read. Total count:  4263
Input count: 4263
Expanded target count: 4263
Merged data count: 4263
Data saved. Total count: 4263
Saved total count:  4263
Data read. Total count:  4491
Input count: 4491
Expanded target count: 4491
Merged data count: 4491
Data saved. Total count: 4491
Saved total count:  4491
Data read. Total count:  4374
Input count: 4374
Expanded target count: 4374
Merged data count: 4374
Data saved. Total count: 4374
Saved total count:  4374
Data read. Total count:  2163
Input count: 2163
Expanded target count: 2163
Merged data count: 2163
Data saved. Total count: 2163
Saved total count:  2163
/========= #2 Read data =========/
Data read. Total count:  834
Data read. Total count:  606
Data read. Total count:  723
Data read. Total count:  2934
/========= #2 Saved data =========/
Data

In [31]:
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]

for k in range(1,11):

    inputs =[]
    
    print(f'/========= #{k} Read data =========/')
    for index, task_dataset in enumerate(task_datasets):
        task, data = task_dataset[0], task_dataset[1]
        source_folder = f'/home/elicer/LLMSS/outputs/{task}/{data}'
        file_name = f'zero_dev_{task}_{data}_{k}.txt'
        _, input = split_sharp(os.path.join(source_folder, file_name))
        inputs.append(input)
    
    print(f'/========= #{k} Saved data =========/')
    for index, task_dataset in enumerate(task_datasets):
        task, data = task_dataset[0], task_dataset[1]  
        temp_list = []
        for i, input_list in enumerate(inputs):
            if index != i: 
                temp_list = temp_list + input_list
    
        source_folder = f'/home/elicer/ATOSS/data/sft/{task}/{data}'
        file_name = f'dev.txt'
        _, targets= split_sharp(os.path.join(source_folder, file_name))
        merge_list = merge_sharp(temp_list, targets, num=1)
        
        source_folder = f'/home/elicer/ABSA/data/{task}/{data}'
        file_name = f'dev_zero_{k}.txt'
        save(os.path.join(source_folder, file_name), merge_list)



/========= #1 Read data =========/
Data read. Total count:  209
Data read. Total count:  277
Data read. Total count:  133
Data read. Total count:  326
/========= #1 Saved data =========/
Data read. Total count:  736
Input count: 736
Expanded target count: 736
Merged data count: 736
Data saved. Total count: 736
Saved total count:  736
Data read. Total count:  668
Input count: 668
Expanded target count: 668
Merged data count: 668
Data saved. Total count: 668
Saved total count:  668
Data read. Total count:  812
Input count: 812
Expanded target count: 812
Merged data count: 812
Data saved. Total count: 812
Saved total count:  812
Data read. Total count:  619
Input count: 619
Expanded target count: 619
Merged data count: 619
Data saved. Total count: 619
Saved total count:  619
/========= #2 Read data =========/
Data read. Total count:  209
Data read. Total count:  277
Data read. Total count:  133
Data read. Total count:  326
/========= #2 Saved data =========/
Data read. Total count:  736
I